<a href="https://colab.research.google.com/github/2021-FIIT-Bc-projects/BP-Eliseeva-Najdiskutovanejsie_temy_v_dokumentoch/blob/main/Bakalarka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import gensim
import nltk
import pandas as pd
import numpy as np
import re
import collections
import itertools

from gensim import corpora
from sklearn.datasets import fetch_20newsgroups
from wordcloud import WordCloud
from nltk.corpus import stopwords
from pprint import pprint
from gensim.models import CoherenceModel

import matplotlib.pyplot as plt
import spacy

In [37]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['bhjn', 'a', 'able', 'about', 'above', 'abst', 'accordance', 'according', 'accordingly', 'across', 'act', 'actually', 'added', 'adj', 'after', 'afterwards', 'again', 'against', 'ah', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anymore', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apparently', 'are', 'aren', 'arent', 'arise', 'around', 'as', 'aside', 'ask', 'asking', 'at', 'auth', 'awfully', 'b', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'begin', 'beginning', 'beginnings', 'begins', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'biol', 'both', 'briefly', 'but', 'by', 'c', 'ca', 'came', 'can', 'cannot', 'cant', 'co', 'com', 'come', 'comes', 'could', 'couldnt', 'd', 'date', 'did', 'didnt', 'different', 'do', 'does', 'doesnt', 'doing', 'done', 'dont', 'down', 'downwards', 'due', 'during', 'e', 'each', 'ed', 'edu', 'eg', 'eight', 'eighty', 'either', 'else', 'elsewhere', 'end', 'ending', 'enough', 'et', 'etc', 'even', 'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'ex', 'except', 'f', 'far', 'few', 'ff', 'fifth', 'first', 'five', 'fix', 'followed', 'following', 'follows', 'for', 'former', 'formerly', 'forth', 'found', 'four', 'from', 'further', 'furthermore', 'g', 'gave', 'get', 'gets', 'getting', 'give', 'given', 'gives', 'giving', 'go', 'goes', 'gone', 'got', 'gotten', 'h', 'had', 'happens', 'hardly', 'has', 'hasnt', 'have', 'havent', 'having', 'he', 'hed', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'heres', 'hereupon', 'hers', 'herself', 'hes', 'hi', 'hid', 'him', 'himself', 'his', 'hither', 'home', 'how', 'howbeit', 'however', 'hundred', 'i', 'id', 'ie', 'if', 'im', 'immediate', 'in', 'inc', 'indeed', 'index', 'instead', 'into', 'invention', 'inward', 'is', 'isnt', 'it', 'itd', 'itll', 'its', 'itself', 'j', 'just', 'k', 'keep',	'keeps', 'kept', 'kg', 'km', 'know', 'known', 'knows', 'l', 'largely', 'last', 'lately', 'later', 'latter', 'latterly', 'least', 'less', 'lest', 'let', 'lets', 'like', 'liked', 'likely', 'line', 'little', 'll', 'look', 'looking', 'looks', 'ltd', 'm', 'made', 'mainly', 'make', 'makes', 'many', 'may', 'maybe', 'me', 'mean', 'means', 'meantime', 'meanwhile', 'merely', 'mg', 'might', 'million', 'miss', 'ml', 'more', 'moreover', 'most', 'mostly', 'mr', 'mrs', 'much', 'mug', 'must', 'my', 'myself', 'n', 'na', 'name', 'namely', 'nay', 'nd', 'near', 'nearly', 'need', 'needs', 'neither', 'never', 'nevertheless', 'new', 'next', 'nine', 'ninety', 'no', 'nobody', 'non', 'none', 'nonetheless', 'noone', 'nor', 'nos', 'not', 'nothing', 'now', 'nowhere', 'o', 'obviously', 'of', 'off', 'often', 'oh', 'ok', 'okay', 'old', 'on', 'once', 'one', 'ones', 'only', 'onto', 'or', 'ord', 'other', 'others', 'otherwise', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'overall', 'owing', 'own', 'p', 'page', 'pages', 'part', 'past', 'per', 'perhaps', 'placed', 'please', 'plus', 'pp', 'predominantly', 'previously', 'probably', 'promptly', 'put', 'q', 'que', 'quickly', 'quite', 'qv', 'r', 'ran', 'rather', 'rd', 're', 'readily', 'really', 'ref', 'refs', 'regarding', 'regardless', 'regards', 'right', 'run', 's', 'said', 'same', 'saw', 'say', 'saying', 'says', 'sec', 'see', 'seeing', 'seem', 'seemed', 'seeming', 'seems', 'seen', 'self', 'selves', 'sent', 'seven', 'shall', 'she', 'shed', 'shell', 'shes', 'should', 'shouldnt', 'show', 'showed', 'shown', 'showns', 'shows', 'since', 'six', 'slightly', 'so', 'some', 'somebody', 'somehow', 'someone', 'somethan', 'something', 'sometime', 'sometimes', 'somewhat', 'somewhere', 'soon', 'still', 'stop', 'strongly', 'sub', 'such', 'sufficiently', 'suggest', 'sup', 'sure	t', 'take', 'taken', 'taking', 'tell', 'tends', 'th', 'than', 'thank', 'thanks', 'thanx', 'that', 'thatll', 'thats', 'thatve', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'thered', 'therefore', 'therein', 'therell', 'thereof', 'therere', 'theres', 'thereto', 'thereupon', 'thereve', 'these', 'they', 'theyd', 'theyll', 'theyre', 'theyve', 'think', 'this', 'those', 'thou', 'though', 'thoughh', 'thousand', 'throug', 'through', 'throughout', 'thru', 'thus', 'til', 'tip', 'to', 'together', 'too', 'took', 'toward', 'towards', 'tried', 'tries', 'try', 'trying', 'ts', 'twice', 'two', 'u', 'un', 'under', 'unfortunately', 'unless', 'unlike', 'unlikely', 'until', 'unto', 'up', 'upon', 'ups', 'us', 'use', 'used', 'useful', 'usefully', 'usefulness', 'uses', 'using', 'usually', 'v', 'value', 've', 'very', 'via', 'viz', 'vol', 'vols', 'vs', 'w', 'want', 'wants', 'was', 'wasnt', 'way', 'we', 'wed', 'well', 'went', 'were', 'werent', 'weve', 'what', 'whatever', 'whatll', 'whats', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', 'wheres', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whim', 'whither', 'who', 'whod', 'whoever', 'whole', 'wholl', 'whom', 'whomever', 'whos', 'whose', 'why', 'widely', 'willing', 'wish', 'with', 'within', 'without', 'wont', 'words', 'would', 'wouldnt', 'www', 'x', 'y', 'yes', 'yet', 'you', 'youd', 'youll', 'your', 'youre', 'yours', 'yourself', 'yourselves', 'youve', 'z', 'zero'])
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
def ShowWordCloud(data):
    long_string = ','.join(list(data.values))
    # Create a WordCloud object
    wordcloud = WordCloud(background_color="white", max_words=5000, max_font_size=60).generate(long_string)

    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()


def sent_to_words(sentences):
    for sentence in sentences:
        yield gensim.utils.simple_preprocess(str(sentence), deacc=True)


def remove_stopwords_spacy(texts):
  filtered_texts = []
  for text in texts:
    filtered_sentence = [] 
    for word in text:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word)
    filtered_texts.append(filtered_sentence)
  return filtered_texts

def remove_stopwords_nltk(texts):
  return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def twenty_newsgroup_to_list():
    newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
    data = {'texts': newsgroups_train.data, 'topic_names': newsgroups_train['target_names'], 'topics_assigned': newsgroups_train['target']}
    return data

In [39]:
papers = twenty_newsgroup_to_list()
papers['texts_processed'] = [re.sub('[ \t\n\r\f\v]', " ", text) for text in papers['texts']]
papers['texts_processed'] = [re.sub(r"\S*@\S*\s?", "", text) for text in papers['texts_processed']]
# print(papers['texts'][2])
# print(papers['texts_processed'][2])
data_words = list(sent_to_words(papers['texts_processed']))
data_words_nostops = remove_stopwords_spacy(data_words)
# print(data_words[2])
print(data_words_nostops[2])
print(remove_stopwords_nltk(data_words)[2])

['folks', 'mac', 'plus', 'finally', 'gave', 'ghost', 'weekend', 'starting', 'life', 'way', 'sooo', 'market', 'new', 'machine', 'bit', 'sooner', 'intended', 'looking', 'picking', 'powerbook', 'maybe', 'bunch', 'questions', 'hopefully', 'somebody', 'answer', 'anybody', 'know', 'dirt', 'round', 'powerbook', 'introductions', 'expected', 'heard', 'supposed', 'appearence', 'summer', 'haven', 'heard', 'anymore', 'don', 'access', 'macleak', 'wondering', 'anybody', 'info', 'anybody', 'heard', 'rumors', 'price', 'drops', 'powerbook', 'line', 'like', 'ones', 'duo', 'went', 'recently', 'impression', 'display', 'probably', 'swing', 'got', 'mb', 'disk', 'don', 'feel', 'better', 'display', 'yea', 'looks', 'great', 'store', 'wow', 'good', 'solicit', 'opinions', 'people', 'use', 'day', 'day', 'worth', 'taking', 'disk', 'size', 'money', 'hit', 'active', 'display', 'realize', 'real', 'subjective', 'question', 've', 'played', 'machines', 'computer', 'store', 'breifly', 'figured', 'opinions', 'somebody', '

In [44]:
bigram = gensim.models.Phrases(data_words_nostops)

bigram_mod = gensim.models.phrases.Phraser(bigram)
data_bigrams = [bigram_mod[text] for text in data_words_nostops]

In [45]:
data_lemmatized = []
for text in data_bigrams:
    doc = nlp(" ".join(text))
    data_lemmatized.append([token.lemma_ for token in doc if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV']])

In [46]:
id2word = corpora.Dictionary(data_lemmatized)

print(len(id2word))
id2word.filter_extremes(no_above=0.5, no_below=5)
print(len(id2word))

corpus = [id2word.doc2bow(text) for text in data_lemmatized]

34359
8844


In [49]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10)
pprint(lda_model.print_topics())
#perplexity
print(lda_model.log_perplexity(corpus))
#coherence
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
print(coherence_model_lda.get_coherence())

[(0,
  '0.024*"send" + 0.023*"thank" + 0.023*"list" + 0.023*"post" + '
  '0.019*"include" + 0.018*"mail" + 0.018*"information" + 0.017*"copy" + '
  '0.016*"source" + 0.015*"line"'),
 (1,
  '0.032*"law" + 0.022*"book" + 0.022*"issue" + 0.017*"die" + 0.017*"case" + '
  '0.016*"public" + 0.013*"state" + 0.013*"person" + 0.012*"concern" + '
  '0.012*"self"'),
 (2,
  '0.031*"use" + 0.025*"system" + 0.023*"window" + 0.021*"card" + 0.019*"need" '
  '+ 0.018*"bit" + 0.017*"problem" + 0.016*"run" + 0.015*"set" + '
  '0.015*"machine"'),
 (3,
  '0.026*"kill" + 0.022*"discussion" + 0.021*"say" + 0.016*"village" + '
  '0.014*"woman" + 0.013*"soldier" + 0.012*"terrorist" + 0.012*"live" + '
  '0.012*"come" + 0.011*"attack"'),
 (4,
  '0.023*"point" + 0.020*"reason" + 0.020*"mean" + 0.016*"think" + '
  '0.015*"argument" + 0.014*"explain" + 0.014*"exist" + 0.013*"thing" + '
  '0.013*"say" + 0.012*"question"'),
 (5,
  '0.807*"ax_ax" + 0.013*"ax" + 0.009*"ad" + 0.007*"rock" + 0.007*"dream" + '
  '0.006*"b